### Download library mediapipe and openCV 

In [1]:
# Download mediapipe
pip install mediapipe

# Download opencv-python
pip install opencv-python

SyntaxError: invalid syntax (354474619.py, line 2)

### Import library

In [1]:
import cv2
import mediapipe as mp
import pygame
import time
import random
import cvzone

pygame 2.1.2 (SDL 2.0.18, Python 3.10.8)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
class handTracker():
    def __init__(self, mode=False, maxHands=2, detectionCon=0.5,modelComplexity=1,trackCon=0.5):
        self.mode = mode
        self.maxHands = maxHands
        self.detectionCon = detectionCon
        self.modelComplex = modelComplexity
        self.trackCon = trackCon
        self.mpHands = mp.solutions.hands
        self.hands = self.mpHands.Hands(self.mode, self.maxHands,self.modelComplex,
                                        self.detectionCon, self.trackCon)
        self.mpDraw = mp.solutions.drawing_utils
        self.sound = "E:\ThisPC\Desktop\Other\DoAnCoSo4\Music\music{}.mp3"
        self.tipIds = [4, 8, 12, 16, 20]
        
    def handsFinder(self,image,draw=True):
        imageRGB = cv2.cvtColor(image,cv2.COLOR_BGR2RGB)
        self.results = self.hands.process(imageRGB)

        if self.results.multi_hand_landmarks:
            for handLms in self.results.multi_hand_landmarks:

                if draw:
                    self.mpDraw.draw_landmarks(image, handLms, self.mpHands.HAND_CONNECTIONS)
        return image
    
    def positionFinder(self,image, handNo=0, draw=True):
        lmlist = []
        if self.results.multi_hand_landmarks:
            Hand = self.results.multi_hand_landmarks[handNo]
            for id, lm in enumerate(Hand.landmark):
                h,w,c = image.shape
                cx,cy = int(lm.x*w), int(lm.y*h)
                lmlist.append([id,cx,cy])
            if draw:
                cv2.circle(image,(cx,cy), 15 , (255,0,255), cv2.FILLED)

        return lmlist
    
    def fingersUp(self, lmList):
        fingers=[]

        # Thumb
        if lmList[self.tipIds[0]][1] > lmList[self.tipIds[0] - 1][1]: 
            fingers.append(1)
        else:
            fingers.append(0)

        for id in range(1,5):  #y axis
            if lmList[self.tipIds[id]][2] < lmList[self.tipIds[id]-2][2]:
                fingers.append(1)
            else:
                fingers.append(0)

        return fingers
    
    def speak(self , num):
        pygame.mixer.init()
        sound = pygame.mixer.music.load(self.sound.format(num))
        pygame.mixer.music.play()
        

In [3]:
class fingerRecognition():
    def main(self):
        cap = cv2.VideoCapture(0)
        tracker = handTracker()
        voice = []
        while True:
            success,image = cap.read()
            image = tracker.handsFinder(image)
            lmList = tracker.positionFinder(image)

            if lmList != []:
                num = tracker.fingersUp(lmList).count(1)
                voice.append(num)
                if len(voice) == 1:
                    tracker.speak(num)
                else:
                    if voice[1] != voice[0]:
                        tracker.speak(voice[1])
                    del voice[0]


            cv2.imshow("Fingers Counting",image)
            cv2.waitKey(1)

            if not cv2.getWindowProperty('Fingers Counting', cv2.WND_PROP_VISIBLE):
                break

In [4]:
if __name__ == "__main__":
    fingerRecognition().main()

In [5]:
class Rock_Paper_Scissors():
    def __init__(self, timer = 0 ,stateResult = False,startGame = False,scores = [0, 0]):
        self.timer = timer
        self.stateResult = stateResult
        self.startGame = startGame
        self.scores = scores
        self.pathImgBG = "E:\ThisPC\Desktop\Other\DoAnCoSo4\Resources\BG.png"
        self.imgAI = "E:\ThisPC\Desktop\Other\DoAnCoSo4\Resources\{}.png"
        
    def main(self):
        cap = cv2.VideoCapture(0)
        detector = handTracker()
        
        while True:
            imgBG = cv2.imread(self.pathImgBG)
            success, img = cap.read()

            imgScaled = cv2.resize(img, (0, 0), None, 0.875, 0.875)
            imgScaled = imgScaled[:, 80:480]
            
            image = detector.handsFinder(imgScaled)
            lmList = detector.positionFinder(imgScaled)

            if self.startGame:
                if self.stateResult is False:
                    self.timer = time.time() - initialTime
                    cv2.putText(imgBG, str(int(self.timer)), (605, 435), cv2.FONT_HERSHEY_PLAIN, 6, (255, 0, 255), 4)

                if self.timer > 3:
                    self.startGame = False
                    self.stateResult = True
                    self.timer = 0
                    checkResult = True

                if checkResult:
                    checkResult = False
                    player = None
                    fingers = detector.fingersUp(lmList)
                    
                    if fingers == [0, 0, 0, 0, 0]:
                        player = 1
                    if fingers == [1, 1, 1, 1, 1]:
                        player = 2
                    if fingers == [0, 1, 1, 0, 0]:
                        player = 3
                    
                    AI = random.randint(1, 3)
                    imgAI = cv2.imread(self.imgAI.format(AI), cv2.IMREAD_UNCHANGED)
                    imgBG = cvzone.overlayPNG(imgBG, imgAI, (149, 310))

                    # Player Wins
                    if (player == 1 and AI == 3) or (player == 2 and AI == 1) or (player == 3 and AI == 2):
                        self.scores[1] += 1

                    # AI Wins
                    if (player == 3 and AI == 1) or (player == 1 and AI == 2) or (player == 2 and AI == 3):
                        self.scores[0] += 1

    
            imgBG[234:654, 795:1195] = imgScaled

            if self.stateResult:
                imgBG = cvzone.overlayPNG(imgBG, imgAI, (149, 310))

            cv2.putText(imgBG, str(self.scores[0]), (410, 215), cv2.FONT_HERSHEY_PLAIN, 4, (255, 255, 255), 6)
            cv2.putText(imgBG, str(self.scores[1]), (1112, 215), cv2.FONT_HERSHEY_PLAIN, 4, (255, 255, 255), 6)

            cv2.imshow("Rock_Paper_Scissors", imgBG)

            key = cv2.waitKey(1)
            if key == ord('s'):
                self.startGame = True
                initialTime = time.time()
                self.stateResult = False
                checkResult = False
    
            # Tắt chương trình
            if not cv2.getWindowProperty('Rock_Paper_Scissors', cv2.WND_PROP_VISIBLE):
                break
        

In [6]:
if __name__ == "__main__":
    Rock_Paper_Scissors().main()